In [44]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import SVR
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

#Reading datasets
LondonDataset = pd.read_csv("/content/london_weather.csv")
TokyoDataset = pd.read_csv("/content/tokyo_data.csv", encoding='ISO-8859-1')
SeattleDataset = pd.read_csv("/content/seattle-weather.csv")

#Removing columns not used and reordering columns
LondonDataset = LondonDataset.drop(['cloud_cover', 'global_radiation', 'snow_depth', 'mean_temp', 'pressure', 'sunshine'], axis=1)
LondonDataset.rename(columns={'max_temp': "high", 'min_temp': "low"}, inplace=True)
LondonDataset = LondonDataset.reindex(columns=['date', 'high', 'low', 'precipitation'])
LondonDataset['date'] = pd.to_datetime(LondonDataset['date'], format='%Y%m%d')
LondonDataset = LondonDataset.fillna(0)

TokyoDataset.rename(columns={'daylight_hours(hours)': "daylight", 'max_precipitation_10min(mm)': "precipitation", 'low()': "low", 'high()': "high", 'y/m/d': "date" }, inplace=True)
TokyoDataset = TokyoDataset.drop(['daylight'], axis=1)
TokyoDataset = TokyoDataset.iloc[5125:,:]
TokyoDataset = TokyoDataset.astype({'high':'float', 'low':'float', 'precipitation':'float'})

SeattleDataset.rename(columns={'temp_min': "low", 'temp_max': "high"}, inplace=True)
SeattleDataset = SeattleDataset.drop(['wind', 'weather'], axis=1)
SeattleDataset = SeattleDataset.reindex(columns=['date', 'high', 'low', 'precipitation'])
SeattleDataset = SeattleDataset.fillna(0)

In [45]:
YearPlot = TokyoDataset.iloc[:365,:]

#plt.plot(YearPlot['date'], YearPlot['low'])
#plt.show()

#plt.plot(TokyoDataset['date'], TokyoDataset['low'])
#plt.show()
#First Plot shows the weather for a year 
#Second Plot shows the weather's seasonality as it repeats each year

In [46]:
YearPlot = LondonDataset.iloc[:365,:]

#plt.plot(YearPlot['date'], YearPlot['low'])
#plt.show()

#plt.plot(LondonDataset['date'], LondonDataset['low'])
#plt.show()
#First Plot shows the weather for a year 
#Second Plot shows the weather's seasonality as it repeats each year

In [47]:
#Methods and KPIs definition

#Naive Prediction 
#Rule: F[i] = A[i-1]
def naive(Dataset, forcastValue, ForcastColumnName):
  f = [Dataset[forcastValue].values[0]]
  for i in range(1,Dataset.shape[0]):
    f.append(Dataset[forcastValue].values[i-1])
  Dataset[ForcastColumnName] = f

#Simple Mean
#Rule: F[i] = Sum A[0]->A[i-1] / Total
def simplemean(Dataset, forcastValue, ForcastColumnName):
  f = [Dataset[forcastValue].values[0]]
  for i in range(2,Dataset.shape[0]+1):
    f.append(Dataset[forcastValue][:i-1].mean())
  Dataset[ForcastColumnName] = f

#Moving Average
#Rule: f[i] = A[i-1]  + A[i-2] + ,,,,,, + A[i-window size]
def movingaverage(Dataset, forcastValue, ForcastColumnName, window_size):
    moving_averages = [0] * window_size  # Initialize with None values for incomplete windows
    for i in range(window_size, Dataset.shape[0]):
        window = Dataset[forcastValue][i-window_size:i]
        average = sum(window) / window_size
        moving_averages.append(average)
    
    Dataset[ForcastColumnName] = moving_averages

#Weighted Moving Average
# Rule: f[i] = A[i-1]*Wi  + A[i-2]*Wi-1 + ,,,,,, + A[i-window size]*Wi/window size

# Function to calculate weighted moving average
def weightedmovingaverage(Dataset, forcastValue, ForcastColumnName, window_size):
    moving_averages = [0] * window_size
    for i in range(window_size, len(Dataset[forcastValue])):
      sum =0
      x=1 
      total=0
      for j in range(i-window_size,i):
        value=Dataset[forcastValue].values[j]*x
        sum=sum+value
        total +=x
        x = x +1
      average = sum / total  # Use weighted average calculation
      moving_averages.append(average)
    Dataset[ForcastColumnName] = moving_averages

#Exponential Smoothing
#Rule: F[i] = F[i-1] + Alpha(A[i-1] - F[i-1])
def exponentialsmoothing(Dataset, forcastValue, ForcastColumnName, alpha):
  f = [Dataset[forcastValue].values[0]]
  for i in range(1,Dataset.shape[0]):
    fnew = f[i-1] + alpha * (Dataset[forcastValue].values[i-1] -  f[i-1])
    f.append(fnew)
  Dataset[ForcastColumnName] = f

#Exponential Smoothing With Trend Adjustment
#Rule: Forcast[i] = f[i] + T[i]
#Rule: F[i] = alpha(A[i-1]) + (1-alpha)(F[i-1] + T[i-1])
#Rule: T[i] = Beta(F[i]-F[i-1]) + (1-Beta)T[i-1]
def exponentialsmoothingtrendadjustment(Dataset, forcastValue, ForcastColumnName, alpha, Beta):
  f = [Dataset[forcastValue].values[0]/1.5]
  t = [Dataset[forcastValue].values[0]/1.5]
  Forcast = [f[0]+t[0]]
  for i in range(1,Dataset.shape[0]):
    fnew = alpha * (Dataset[forcastValue].values[i-1] + (1-alpha)*(f[i-1] + t[i-1]))
    f.append(fnew)
    tnew = Beta * (f[i]-f[i-1]) + (1-Beta)*t[i-1]
    t.append(tnew)
    Forcast.append(fnew+tnew)
  Dataset[ForcastColumnName] = f

def supportvectorregression(Dataset, forcastValue, ForcastColumnName):
  model = SVR(kernel='linear')
  x = []
  for i in range(len(Dataset)):
    x.append(Dataset.index)
  model.fit(x, Dataset[forcastValue])
  yfit = model.predict(Dataset['low'])
  
#Autoregressive Integrated Moving Average
def arima(Dataset, forcastValue, ForcastColumnName):
  # Creating ARIMA model  
  mymodel = ARIMA(Dataset['low'], order = (1, 1, 1))  
  modelfit = mymodel.fit()  
  
  Dataset[ForcastColumnName] =modelfit.predict() 

def sarima(Dataset, forcastValue, ForcastColumnName):
    # Creating SARIMA model
    model = SARIMAX(Dataset[forcastValue], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    modelfit = model.fit()

    # Perform forecast and add to the dataset
    Dataset[ForcastColumnName] = modelfit.predict(start=Dataset.index.min(), end=Dataset.index.max())

#Mean error
#Rule: Σ(A[i]-f[i])/n
def meanerror(Dataset, actual, forcast):
  sum = 0
  for i in range(0,Dataset.shape[0]):
    sum = sum +(Dataset[actual].values[i] -  Dataset[forcast].values[i])
  return(sum/Dataset.shape[0])

#Mean Absolute deviation
#Rule:
def mad(Dataset, actual, forcast):
  sum = 0
  for i in range(0,Dataset.shape[0]):
    sum = sum +abs((Dataset[actual].values[i] -  Dataset[forcast].values[i]))
  return(sum/Dataset.shape[0])

#Mean Square Error
#Rule:
def mse(Dataset, actual, forcast):
  sum = 0
  for i in range(0,Dataset.shape[0]):
    sum = sum +pow((Dataset[actual].values[i] -  Dataset[forcast].values[i]),2)
  return(sum/Dataset.shape[0])

#Mean Absolute Percent Error
#Rule:
def mape(Dataset, actual, forcast):
  sum = 0
  for i in range(0,Dataset.shape[0]):
    if(Dataset[actual].values[i] != 0):
      sum = sum +abs((Dataset[actual].values[i] -  Dataset[forcast].values[i]))/Dataset[actual].values[i]
    else:
      sum = sum +abs((Dataset[actual].values[i] -  Dataset[forcast].values[i]))/.1
  return(sum/Dataset.shape[0])

#Plot actual value against forcast value
def plotforcast(Dataset, Original, forcast):
  YearPlot = Dataset.iloc[:365,:]
  plt.plot(YearPlot['date'], YearPlot['low'], label = "Original")
  plt.plot(YearPlot['date'], YearPlot['lowpredict'], label = "forcast")
  plt.legend()
  plt.show()

In [48]:
ResultsDataframe = pd.DataFrame(index=['Naive', 'Simple Mean', 'Moving Average','Weighted Moving Average', 'Exponential Smoothing', 'Exponential Smoothing With Trend Adjustment', 'ARIMA', 'SARIMA'] , columns=['Mean error', 'Mean Absolute deviation', 'Mean Square Error', 'Mean Absolute Percent Error'])

In [49]:
#LondonDataset
#TokyoDataset
#SeattleDataset
curdataset = LondonDataset

In [50]:
#Naive
naive(curdataset, 'low', 'lowpredict')

ResultsDataframe['Mean error']['Naive'] = meanerror(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Absolute deviation']['Naive'] = mad(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Square Error']['Naive'] = mse(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Absolute Percent Error']['Naive'] = mape(curdataset, 'low', 'lowpredict')

#plotforcast(curdataset, 'low', 'lowpredict')
ResultsDataframe

,Mean error,Mean Absolute deviation,Mean Square Error,Mean Absolute Percent Error
Naive,0.000287,2.166821,7.658028,0.479718
Simple Mean,NaN,NaN,NaN,NaN
Moving Average,NaN,NaN,NaN,NaN
Weighted Moving Average,NaN,NaN,NaN,NaN
Exponential Smoothing,NaN,NaN,NaN,NaN
Exponential Smoothing With Trend Adjustment,NaN,NaN,NaN,NaN
ARIMA,NaN,NaN,NaN,NaN
SARIMA,NaN,NaN,NaN,NaN


In [51]:
#Simple Mean
simplemean(curdataset, 'low', 'lowpredict')

ResultsDataframe['Mean error']['Simple Mean'] = meanerror(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Absolute deviation']['Simple Mean'] = mad(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Square Error']['Simple Mean'] = mse(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Absolute Percent Error']['Simple Mean'] = mape(curdataset, 'low', 'lowpredict')

#plotforcast(curdataset, 'low', 'lowpredict')
ResultsDataframe

,Mean error,Mean Absolute deviation,Mean Square Error,Mean Absolute Percent Error
Naive,0.000287,2.166821,7.658028,0.479718
Simple Mean,0.611172,4.458869,28.390487,0.745692
Moving Average,NaN,NaN,NaN,NaN
Weighted Moving Average,NaN,NaN,NaN,NaN
Exponential Smoothing,NaN,NaN,NaN,NaN
Exponential Smoothing With Trend Adjustment,NaN,NaN,NaN,NaN
ARIMA,NaN,NaN,NaN,NaN
SARIMA,NaN,NaN,NaN,NaN


In [52]:
#Moving Average
movingaverage(curdataset, 'low', 'lowpredict',3)
ResultsDataframe['Mean error']['Moving Average'] = meanerror(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Absolute deviation']['Moving Average'] = mad(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Square Error']['Moving Average'] = mse(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Absolute Percent Error']['Moving Average'] = mape(curdataset, 'low', 'lowpredict')

#plotforcast(curdataset, 'low', 'lowpredict')
ResultsDataframe

,Mean error,Mean Absolute deviation,Mean Square Error,Mean Absolute Percent Error
Naive,0.000287,2.166821,7.658028,0.479718
Simple Mean,0.611172,4.458869,28.390487,0.745692
Moving Average,-0.000671,2.284078,8.393738,0.437567
Weighted Moving Average,NaN,NaN,NaN,NaN
Exponential Smoothing,NaN,NaN,NaN,NaN
Exponential Smoothing With Trend Adjustment,NaN,NaN,NaN,NaN
ARIMA,NaN,NaN,NaN,NaN
SARIMA,NaN,NaN,NaN,NaN


In [53]:
#Weighted Moving Average
weightedmovingaverage(curdataset, 'low', 'lowpredict',5)
ResultsDataframe['Mean error']['Weighted Moving Average'] = meanerror(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Absolute deviation']['Weighted Moving Average'] = mad(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Square Error']['Weighted Moving Average'] = mse(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Absolute Percent Error']['Weighted Moving Average'] = mape(curdataset, 'low', 'lowpredict')

#plotforcast(curdataset, 'low', 'lowpredict')
ResultsDataframe

,Mean error,Mean Absolute deviation,Mean Square Error,Mean Absolute Percent Error
Naive,0.000287,2.166821,7.658028,0.479718
Simple Mean,0.611172,4.458869,28.390487,0.745692
Moving Average,-0.000671,2.284078,8.393738,0.437567
Weighted Moving Average,-0.001432,2.220408,7.886533,0.424733
Exponential Smoothing,NaN,NaN,NaN,NaN
Exponential Smoothing With Trend Adjustment,NaN,NaN,NaN,NaN
ARIMA,NaN,NaN,NaN,NaN
SARIMA,NaN,NaN,NaN,NaN


In [54]:
#Exponential Smoothing
exponentialsmoothing(curdataset, 'low', 'lowpredict',.2)

ResultsDataframe['Mean error']['Exponential Smoothing'] = meanerror(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Absolute deviation']['Exponential Smoothing'] = mad(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Square Error']['Exponential Smoothing'] = mse(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Absolute Percent Error']['Exponential Smoothing'] = mape(curdataset, 'low', 'lowpredict')

#plotforcast(curdataset, 'low', 'lowpredict')
ResultsDataframe

,Mean error,Mean Absolute deviation,Mean Square Error,Mean Absolute Percent Error
Naive,0.000287,2.166821,7.658028,0.479718
Simple Mean,0.611172,4.458869,28.390487,0.745692
Moving Average,-0.000671,2.284078,8.393738,0.437567
Weighted Moving Average,-0.001432,2.220408,7.886533,0.424733
Exponential Smoothing,0.002976,2.252528,8.018451,0.402442
Exponential Smoothing With Trend Adjustment,NaN,NaN,NaN,NaN
ARIMA,NaN,NaN,NaN,NaN
SARIMA,NaN,NaN,NaN,NaN


In [55]:
#Exponential Smoothing With Trend Adjustment
exponentialsmoothingtrendadjustment(curdataset, 'low', 'lowpredict', .8, .2)

ResultsDataframe['Mean error']['Exponential Smoothing With Trend Adjustment'] = meanerror(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Absolute deviation']['Exponential Smoothing With Trend Adjustment'] = mad(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Square Error']['Exponential Smoothing With Trend Adjustment'] = mse(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Absolute Percent Error']['Exponential Smoothing With Trend Adjustment'] = mape(curdataset, 'low', 'lowpredict')

#plotforcast(curdataset, 'low', 'lowpredict')
ResultsDataframe

,Mean error,Mean Absolute deviation,Mean Square Error,Mean Absolute Percent Error
Naive,0.000287,2.166821,7.658028,0.479718
Simple Mean,0.611172,4.458869,28.390487,0.745692
Moving Average,-0.000671,2.284078,8.393738,0.437567
Weighted Moving Average,-0.001432,2.220408,7.886533,0.424733
Exponential Smoothing,0.002976,2.252528,8.018451,0.402442
Exponential Smoothing With Trend Adjustment,0.360347,2.130728,7.334367,0.44791
ARIMA,NaN,NaN,NaN,NaN
SARIMA,NaN,NaN,NaN,NaN


In [56]:
#supportvectorregression(curdataset, 'low', 'lowpredict')


In [ ]:
#ARIMA
arima(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean error']['ARIMA'] = meanerror(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Absolute deviation']['ARIMA'] = mad(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Square Error']['ARIMA'] = mse(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Absolute Percent Error']['ARIMA'] = mape(curdataset, 'low', 'lowpredict')

#plotforcast(curdataset, 'low', 'lowpredict')
ResultsDataframe

In [ ]:
sarima(curdataset, 'low', 'lowpredict')

ResultsDataframe['Mean error']['SARIMA'] = meanerror(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Absolute deviation']['SARIMA'] = mad(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Square Error']['SARIMA'] = mse(curdataset, 'low', 'lowpredict')
ResultsDataframe['Mean Absolute Percent Error']['SARIMA'] = mape(curdataset, 'low', 'lowpredict')

#plotforcast(curdataset, 'low', 'lowpredict')
ResultsDataframe